Read two saved template Models 
Interpolate Ge and Ph data to create input and output targets for training.  

1. Load existing template NN models
2. Create parameters for generating training data from source template models
3. Initialize template interpolation (ti) model
4. Train ti model
5. Generate test data for graphing
6. Save ti model to file
7. Generate blender data from ti model output

In [2]:
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
import numpy as np
import json 
import math as m
import random
import time
import NNArt as gy
from sympy.utilities.iterables import multiset_permutations

import plotly as py
import plotly.offline as offline
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.io as pio

CURRENT_MODEL_SAVE_DIR = './save/'

offline.init_notebook_mode(connected=True) 
np.set_printoptions(precision=3, suppress=True)

In [3]:
class Net(nn.Module):
    def __init__(self, layerSizes, name):
        super(Net, self).__init__()
        self.shape = layerSizes
        self.name = name
        self.seq = nn.Sequential(
            nn.Linear(layerSizes[0],layerSizes[1]),
            nn.ELU(),          
            nn.Linear(layerSizes[1],layerSizes[2]),
            nn.ELU(),            
            nn.Linear(layerSizes[2],layerSizes[3]),
            nn.ELU()
        )          
    def forward(self, x):
        return self.seq(x) 

In [8]:
def phRcPhTrain(relModel,phModel, parms):
    epochs = parms['epochs']
    lrR = parms['lrR']
    lrP = parms['lrP']
    tModels = parms['tModels']
    device = parms['device']
    numSamples = parms['numSamples']
    rFilter = parms['rFilter'] 
    numContextDims=1 #extra emergent dimensions

    minLossP = 1000
    maxLossP = -10
    minLossR = 10
    maxLossR = 0
    criterion = torch.nn.MSELoss(reduction='sum')
    relOptimizer = torch.optim.Adam(relModel.parameters(), lr=lrR)
    phOptimizer = torch.optim.Adam(phModel.parameters(), lr=lrP)
    for epoch in range(epochs):
        #Each epoch get a new training set based on random input to the template models
        relTarget, phTarget, _ = gy.createPhRcTargetsFromModelList(tModels, numSamples, numContextDims)

        phOptimizer.zero_grad()
        relOptimizer.zero_grad()
        
        relOutput = relModel(phTarget)
        phOutput = phModel(relOutput)

        lossP = criterion(phOutput, phTarget)
        lossR = criterion(relOutput*rFilter, relTarget*rFilter)

        (lossP).backward(retain_graph=True)
        (lossR).backward()
        
        phOptimizer.step()
        relOptimizer.step()
        
        minLossP = min(lossP.data.item(), minLossP)
        maxLossP = max(lossP.data.item(), maxLossP)
        minLossR = min(lossR.data.item(), minLossR)
        maxLossR = max(lossR.data.item(), maxLossR)
        
        if (epoch) % (epochs/10) == 0:
            print("Epoch {0:,.0f} - lossR: {1:,.5f} - lossP: {2:,.5f}".format(
                epoch, lossR/numSamples, lossP/numSamples))
            
    print("Epoch {0:,.0f} - lossR: {1:,.5f}:{2:,.5f} - lossP: {3:,.5f}:{4:,.5f}".format(epoch+1, 
                minLossR/numSamples, maxLossR/numSamples, minLossP/numSamples, maxLossP/numSamples))

    return

In [5]:
'''
1. Load existing template NN models
templateModel2

'''

path=CURRENT_MODEL_SAVE_DIR+'templateModel'
tmModel0 = Net([2,10,15,21],'tmModel0')
tmModel1 = Net([2,10,15,21],'tmModel1')

tmModel0.load_state_dict(torch.load(path+'0.pth'))
tmModel1.load_state_dict(torch.load(path+'1.pth'))
tmModel0.eval()
tmModel1.eval()

tmModel0.shape, tmModel1.shape

([2, 10, 15, 21], [2, 10, 15, 21])

In [6]:
'''
2. Initialize P-Gc-P

'''

#initialize network
manualSeed = 1

np.random.seed(manualSeed)
# random.seed(manualSeed)
torch.manual_seed(manualSeed)

relModel = Net([21,15,10,3],'relModel')
phModel = Net([3,10,15,21],'phModel')
relModel.train()
phModel.train()

# device = torch.device("cuda:0")
device = torch.device("cpu")
if device:
    relModel.to(device)
    phModel.to(device)
    tmModel0.to(device)
    tmModel1.to(device)
    print("Transfer to use ", device)

relModel.shape, phModel.shape

Transfer to use  cpu


([21, 15, 10, 3], [3, 10, 15, 21])

In [9]:
''' 
3. Train template interpolation model

'''
start = time.time()

parms={ 'epochs':100000,'lrR':.0001,'lrP':.0002,'numSamples':1,'rFilter':torch.Tensor([1,1,.05]),
       'tModels':[tmModel0,tmModel1],'device':device}
phRcPhTrain(relModel,phModel,parms)

print('time: ', time.time()-start)
relModel.train = False
phModel.train = False

Epoch 0 - lossR: 0.39050 - lossP: 0.67026
Epoch 10,000 - lossR: 0.04482 - lossP: 0.07583
Epoch 20,000 - lossR: 0.05111 - lossP: 0.01110
Epoch 30,000 - lossR: 0.01354 - lossP: 0.00702
Epoch 40,000 - lossR: 0.01232 - lossP: 0.00682
Epoch 50,000 - lossR: 0.00028 - lossP: 0.00394
Epoch 60,000 - lossR: 0.06151 - lossP: 0.00325
Epoch 70,000 - lossR: 0.00077 - lossP: 0.00112
Epoch 80,000 - lossR: 0.00238 - lossP: 0.00059
Epoch 90,000 - lossR: 0.00154 - lossP: 0.00226
Epoch 100,000 - lossR: 0.00000:3.46448 - lossP: 0.00002:6.15427
time:  245.02311754226685


In [ ]:
'''
parms={ 'epochs':100000,'lrG':.0004,'lrP':.0005,'numSamples':1,'gFilter':torch.Tensor([1,1,.05]),
Epoch 100,000 - lossG: 0.00000:0.27628 - lossP: 0.00002:0.07341
parms={ 'epochs':100000,'lrG':.0002,'lrP':.0003,'numSamples':1,'gFilter':torch.Tensor([1,1,.05]),
Epoch 100,000 - lossG: 0.00000:0.23505 - lossP: 0.00001:0.06539
parms={ 'epochs':100000,'lrG':.0001,'lrP':.0002,'numSamples':1,'gFilter':torch.Tensor([1,1,.05]),
Epoch 100,000 - lossG: 0.00000:0.21284 - lossP: 0.00001:0.04594
Epoch 100,000 - lossG: 0.00000:0.21077 - lossP: 0.00001:0.03137   
Epoch 100,000 - lossG: 0.00000:0.19727 - lossP: 0.00001:0.03196
'''
1==1

In [14]:
'''
4. Review Emergent Context 


'''
morphIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]
timeIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]

#sample source templates to get ph samples
relIn = gy.createRmInput(morphIndices, timeIndices, sPermutation=[])
phTout0 = tmModel0(torch.Tensor(relMapIn))
phTout1 = tmModel1(torch.Tensor(relMapIn))

#use template generated ph samples to feed through contex models
relCout0 = relModel(phTout0)
relCout1 = relModel(phTout1)

phCout0 = phModel(relCout0)
phCout1 = phModel(relCout1)

# gin.shape, pout2.shape, geCout2.shape, phCout2.shape

In [16]:
g20 = np.array([relIn[0,:,1],relCout0[0,:,2].detach().numpy()])
g22 = np.array([relIn[2,:,1],relCout0[2,:,2].detach().numpy()])
g24 = np.array([relIn[4,:,1],relCout0[4,:,2].detach().numpy()])
g26 = np.array([relIn[6,:,1],relCout0[6,:,2].detach().numpy()])
g27 = np.array([relIn[7,:,1],relCout0[7,:,2].detach().numpy()])
g28 = np.array([relIn[8,:,1],relCout0[8,:,2].detach().numpy()])

g30 = np.array([relIn[0,:,1],relCout1[0,:,2].detach().numpy()])
g32 = np.array([relIn[2,:,1],relCout1[2,:,2].detach().numpy()])
g34 = np.array([relIn[4,:,1],relCout1[4,:,2].detach().numpy()])
g36 = np.array([relIn[6,:,1],relCout1[6,:,2].detach().numpy()])
g37 = np.array([relIn[7,:,1],relCout1[7,:,2].detach().numpy()])
g38 = np.array([relIn[8,:,1],relCout1[8,:,2].detach().numpy()])

gx = [g20,g30,g22,g32,g24,g34,g26,g36,g28,g38]


In [17]:
layout = go.Layout(
    title = 'Initial Curves',
    height = 700,
    width = 800,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.1,1.5],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.6,1.],
    )
)
layout.title = 'PhC changes into GeOutput Sp Space for GeInput: ' 
f = gy.graphCurves(gx, layout, labels=['s2t0', 's3t0','s2t2', 's3t2','s2t4', 's3t4','s2t6', 's3t6','s2t8', 's3t8', ], dashRange=gy.rangeList([range(2,4),range(6,8)]))

offline.iplot(f)


In [18]:

i = 5
p0 = phTout0.data.numpy()[i]
p1 = phTout1.data.numpy()[i]

p01_0 = phCout0.data.numpy()[i]
p01_1 = phCout1.data.numpy()[i]

# nx = np.concatenate((p0,p01_0))
nx = np.concatenate((p1,p01_1))

gx = gy.FormatNetworkToGraphColumns(nx, xOffset=0, yOffset=14, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.4,1.2],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.4,1.2],
    )
)
f = gy.graphCurves(gx, layout, dashRange=range(0,9))
offline.iplot(f)


In [19]:

i = 7
p0 = phTout0.data.numpy()[i]
p1 = phTout1.data.numpy()[i]

p01_0 = phCout0.data.numpy()[i]
p01_1 = phCout1.data.numpy()[i]

# nx = np.concatenate((p0,p01_0))
nx = np.concatenate((p1,p01_1))

gx = gy.FormatNetworkToGraphColumns(nx, xOffset=0, yOffset=7, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.3,1.1],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.2,.2],
    )
)
f = gy.graphCurves(gx, layout, dashRange=range(0,9))
offline.iplot(f)


In [20]:

i = 7
p0 = phTout0.data.numpy()[i]
p1 = phTout1.data.numpy()[i]

p01_0 = phCout0.data.numpy()[i]
p01_1 = phCout1.data.numpy()[i]

# nx = np.concatenate((p0,p01_0))
nx = np.concatenate((p1,p01_1))


# gx = gy.FormatNetworkToGraphColumns(nx, xOffset=0, yOffset=7, colPoints=7)
# gx = gy.FormatNetworkToGraphColumns(nx, xOffset=0, yOffset=14, colPoints=7)
gx = gy.FormatNetworkToGraphColumns(nx, xOffset=7, yOffset=14, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.5,.5],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.8,.2],
    )
)
f = gy.graphCurves(gx, layout, dashRange=range(0,9))
offline.iplot(f)


In [24]:
'''
5. Graph data 
'''
ti = 7
nx = np.array(phTout1[ti].data.numpy())
gx = gy.FormatNetworkToGraphColumns(nx, xOffset=0, yOffset=7, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.4,1.1],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.25,.25],
    )
)

layout.title = 'Time 0 for different species combinations'
f = gy.graphCurves(gx, layout)
offline.iplot(f)

In [25]:
'''
7. Save two template interpolation model to file

'''

# PATH = CURRENT_MODEL_SAVE_DIR +'PH PhRcPh-T0T1-mask 11p05.pth'
# torch.save(phModel.state_dict(), PATH)
# PATH = CURRENT_MODEL_SAVE_DIR +'REL PhRcPh-T0T1-mask 11p05.pth'
# torch.save(geModel.state_dict(), PATH)


'\n7. Save two template interpolation model to file\n\n'

In [26]:
'''
9. Generate blender data from template interpolation model output

#phOut.shape torch.Size([10, 7, 23])

'''

# phOut = phModel(torch.Tensor(torch.Tensor(inpt0)))

# x = phOut.detach().numpy()
# fb = gy.FormatNetworkToBlender(x, xOffset=0, yOffset=7, zOffset=14, colPoints=7)

# with open(CURRENT_MODEL_SAVE_DIR +'btest.json', 'w') as fp:
#     json.dump(fb.tolist(),fp)
    
# x.shape, fb.shape

'\n9. Generate blender data from template interpolation model output\n\n#phOut.shape torch.Size([10, 7, 23])\n\n'